In [ ]:
# Setup: imports and constants
import os, glob, zipfile
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
DATA_DIR = "/workspace/data"
OUTPUT_SUBMISSION = "/workspace/submission.csv"
OUTPUT_ZIP = "/workspace/result.zip"
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# Load and align
from typing import Dict, Tuple
def infer_columns(df: pd.DataFrame):
    lower_cols = {c.lower(): c for c in df.columns}
    ts_col = None
    for key in lower_cols:
        if "time" in key or key in {"timestamp", "datetime"}:
            ts_col = lower_cols[key]; break
    if ts_col is None: ts_col = df.columns[0]
    numeric_cols = [c for c in df.columns if c != ts_col and pd.api.types.is_numeric_dtype(df[c])]
    val_col = numeric_cols[0] if numeric_cols else [c for c in df.columns if c != ts_col][0]
    return ts_col, val_col
def read_sensor_csv(path: str):
    name = os.path.basename(path).replace("_test.csv", "")
    df = pd.read_csv(path)
    ts_col, val_col = infer_columns(df)
    df[ts_col] = pd.to_datetime(df[ts_col], errors="coerce")
    df = df.dropna(subset=[ts_col]).sort_values(ts_col).reset_index(drop=True)
    df = df[[ts_col, val_col]].rename(columns={ts_col: "timestamp", val_col: name})
    return name, df
csv_files = sorted(glob.glob(os.path.join(DATA_DIR, "*_test.csv")))
sensor_to_df, sensor_lengths = {}, {}
for f in csv_files:
    s, sdf = read_sensor_csv(f); sensor_to_df[s]=sdf; sensor_lengths[s]=len(sdf)
highest_sensor = max(sensor_lengths.items(), key=lambda kv: kv[1])[0]
base_timeline = sensor_to_df[highest_sensor][["timestamp"]].dropna().drop_duplicates().sort_values("timestamp").reset_index(drop=True)
aligned = base_timeline.copy()
for s, sdf in sensor_to_df.items():
    sdf = sdf.sort_values("timestamp").reset_index(drop=True)
    merged = pd.merge_asof(base_timeline.sort_values("timestamp"), sdf, on="timestamp", direction="nearest", tolerance=pd.Timedelta("5min"))
    aligned[s] = merged[s]
aligned = aligned.sort_values("timestamp").reset_index(drop=True)
value_cols = [c for c in aligned.columns if c != "timestamp"]
aligned[value_cols] = aligned[value_cols].ffill().bfill()
print(f"Loaded {len(csv_files)} sensors. Highest-rate sensor: {highest_sensor}. Timeline length: {len(aligned)}")

In [ ]:
# Feature engineering
ROLL_WINDOWS = [5, 15, 60]; NUM_LAGS = 3
fe = aligned.copy()
for col in value_cols:
    for w in ROLL_WINDOWS:
        r = fe[col].rolling(window=w, min_periods=max(2, w//3))
        fe[f"{col}_rollmean_{w}"] = r.mean(); fe[f"{col}_rollstd_{w}"] = r.std()
        fe[f"{col}_rollmin_{w}"] = r.min(); fe[f"{col}_rollmax_{w}"] = r.max()
for col in value_cols:
    for lag in range(1, NUM_LAGS+1):
        fe[f"{col}_lag_{lag}"] = fe[col].shift(lag)
feature_cols = [c for c in fe.columns if c != "timestamp"]
fe[feature_cols] = fe[feature_cols].bfill().ffill()
scaler = StandardScaler(); X = scaler.fit_transform(fe[feature_cols])
print(X.shape)

In [ ]:
# Train and predict
iso = IsolationForest(n_estimators=300, contamination=0.01, random_state=RANDOM_STATE, n_jobs=-1)
iso.fit(X)
scores = iso.score_samples(X)
labels = (iso.predict(X)==-1).astype(int)
pred_df = pd.DataFrame({"timestamp": fe["timestamp"].values, "anomaly_score": scores, "prediction": labels})
pred_df.head()

In [ ]:
# Save submission and zip
submission = pd.DataFrame({"prediction": pred_df["prediction"].astype(int).values})
assert len(submission) == len(aligned)
submission.to_csv(OUTPUT_SUBMISSION, index=False)
with zipfile.ZipFile(OUTPUT_ZIP, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write(OUTPUT_SUBMISSION, arcname=os.path.basename(OUTPUT_SUBMISSION))
    zf.write("/workspace/notebook.ipynb", arcname="notebook.ipynb")
print(f"Wrote {OUTPUT_SUBMISSION} and {OUTPUT_ZIP}")